In [7]:
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
import wandb

accuracy = evaluate.load("accuracy")
print("Hi")

import transformers

print(transformers.__version__)



Hi
4.28.1


In [2]:
ds = load_dataset("bamertl/nlp_processed")


Found cached dataset parquet (/home/lub/.cache/huggingface/datasets/bamertl___parquet/bamertl--nlp_processed-c6ca71bcf071c95c/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [4]:
model_name = "xlm-roberta-base"
index = "1"
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=2, id2label=id2label, label2id=label2id
).to("cuda")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

In [5]:
def preprocess_batched(examples):
    elements = tokenizer(examples["text1"], examples["text2"], return_token_type_ids=True,is_split_into_words=True, truncation=True)
    elements["labels"] = examples["has_change"]
    return elements

def preprocess(example):
    inputs = tokenizer(example["text1"], example["text2"], return_token_type_ids=True,is_split_into_words=True, truncation=True)
    inputs["labels"] = example["has_change"]
    return inputs
ds_processed = ds.map(preprocess)

Loading cached processed dataset at /home/lub/.cache/huggingface/datasets/bamertl___parquet/bamertl--nlp_processed-c6ca71bcf071c95c/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-69cf89165f824467.arrow
Loading cached processed dataset at /home/lub/.cache/huggingface/datasets/bamertl___parquet/bamertl--nlp_processed-c6ca71bcf071c95c/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-fee6b339fe4e5112.arrow


In [6]:
wandb.init(project= "nlp-deep", name=model_name + index)
training_args = TrainingArguments(
    output_dir=model_name + index,
    learning_rate=2e-5,
    report_to="tensorboard",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=5,
    logging_steps=5,
    save_total_limit=5,
    load_best_model_at_end=True,
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_processed["train"],
    eval_dataset=ds_processed["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,

)
trainer.train()

wandb: Currently logged in as: bamertl (damian_luk). Use `wandb login --relogin` to force relogin


/home/lub/Projects/ml_challenge/venv/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy
5,0.688100,0.693360,0.474040


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:27                                                                                   │
│                                                                                                  │
│   24 │   compute_metrics=compute_metrics,                                                        │
│   25                                                                                             │
│   26 )                                                                                           │
│ ❱ 27 trainer.train()                                                                             │
│   28                                                                                             │
│                                                                                                  │
│ /home/lub/Projects/ml_challenge/venv/lib/python3.10/site-packages/transformers/trainer.py:1662   │
│ in train                                                                                         │
│                                                                                                  │
│   1659 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1660 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1661 │   │   )                                                                                 │
│ ❱ 1662 │   │   return inner_training_loop(                                                       │
│   1663 │   │   │   args=args,                                                                    │
│   1664 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1665 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /home/lub/Projects/ml_challenge/venv/lib/python3.10/site-packages/transformers/trainer.py:2006   │
│ in _inner_training_loop                                                                          │
│                                                                                                  │
│   2003 │   │   │   │   │   self.state.epoch = epoch + (step + 1 + steps_skipped) / steps_in_epo  │
│   2004 │   │   │   │   │   self.control = self.callback_handler.on_step_end(args, self.state, s  │
│   2005 │   │   │   │   │                                                                         │
│ ❱ 2006 │   │   │   │   │   self._maybe_log_save_evaluate(tr_loss, model, trial, epoch, ignore_k  │
│   2007 │   │   │   │   else:                                                                     │
│   2008 │   │   │   │   │   self.control = self.callback_handler.on_substep_end(args, self.state  │
│   2009                                                                                           │
│                                                                                                  │
│ /home/lub/Projects/ml_challenge/venv/lib/python3.10/site-packages/transformers/trainer.py:2287   │
│ in _maybe_log_save_evaluate                                                                      │
│                                                                                                  │
│   2284 │   │   │   │   │   )                                                                     │
│   2285 │   │   │   │   │   metrics.update(dataset_metrics)                                       │
│   2286 │   │   │   else:                                                                         │
│ ❱ 2287 │   │   │   │   metrics = self.evaluate(ignore_keys=ignore_keys_for_eval)                 │
│   2288 │   │   │   self._report_to_hp_search(trial, self.state.global_step, metrics)             │
│   2289 │   │                                               